# Dados abertos CAPES - Modelagem para painel do GID

In [1]:
#Importando bibliotecas necessárias:
import os
import time
import re
import ssl
import requests
from urllib.parse import urlparse
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import json
import numpy as np

In [2]:
#Diretórios usados para armazenar os dados (baixados e processados)
dirs = {
    'download_dir': 'capes_csv_files',
    'filtered_dir': 'ufrj_data',
    'processed_dir': 'sucupira_painel',
    'discentes': 'discentes',
    'docentes': 'docentes',
    'programas': 'programas',
    'cursos':  'programas',
    'producao': 'producao',
    'producao_detalhe': 'producao',
    'producao_autor': 'producao',
    'projetos': 'projetos',
    'membros': 'projetos',
    'financiadores': 'financiadores',
    'btd': 'btd',
}

#Base directories
download_dir = dirs.get('download_dir')
filtered_dir = dirs.get('filtered_dir')
processed_dir = dirs.get('processed_dir')

## Download dos dados abertos da CAPES

In [ ]:
# Configurações do download
api_url = "https://dadosabertos.capes.gov.br/api/3/action/package_search"
organization = "diretoria-de-avaliacao"
output_dir = dirs.get('download_dir', 'capes_csv_files')
timeout_seconds = 30
max_retries = 3


prefix_substring_dirs = {
    "ddi-br-capes-colsucup-": {
        "projeto-financiador": dirs.get("financiadores", "financiadores"),
        "projeto": dirs.get("projetos", "projetos"),
    }, 
    "br-capes-colsucup-": {
        "prod": dirs.get("producao", "producao"),
        "producao": dirs.get("producao", "producao"),
        "projeto": dirs.get("projetos", "projetos"),
        "membro": dirs.get("projetos", "projetos"),
        "prog": dirs.get("programas", "programas"),
        "curso": dirs.get("cursos", "cursos"),
		"discentes": dirs.get("discentes", "discentes"),
		"docente": dirs.get("docentes", "docentes"),
        "financiador": dirs.get("financiadores", "financiadores"),
    },
    "br-capes-col-": {
        "proj": dirs.get("projetos", "projetos"),
        "producao": dirs.get("producao", "producao"),
        "prod": dirs.get("producao", "producao"),
    },
	"br-colsucup-": {
		"prod": dirs.get("producao", "producao"),
	},
    "br-capes-btd-": {
        "": dirs.get("btd", "btd")
    },
} # Mapeia prefix+substring para pastas (diretórios)

# Sessão com retry
session = requests.Session()
retry = Retry(total=max_retries, backoff_factor=1)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Contexto SSL customizado (caso precise ignorar erros SSL)
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Criar diretório base
os.makedirs(output_dir, exist_ok=True)

def sanitize_filename(filename):
    """Remove caracteres inválidos para nomes de arquivo"""
    return re.sub(r'[\\/*?:"<>|]', "_", filename)

def get_with_retry(url):
    """Faz uma requisição HTTP com retry e fallback para SSL desabilitado ou HTTP"""
    for attempt in range(max_retries):
        try:
            print(f"Tentativa {attempt + 1} para {url}")
            try:
                response = session.get(url, timeout=timeout_seconds)
                response.raise_for_status()
                return response
            except requests.exceptions.SSLError:
                print("Falha SSL, tentando com verificação desativada...")
                response = session.get(url, timeout=timeout_seconds, verify=False)
                response.raise_for_status()
                return response
        except requests.exceptions.RequestException as e:
            print(f"Erro na tentativa {attempt + 1}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
    if url.startswith('https://'):
        http_url = url.replace('https://', 'http://', 1)
        print(f"Tentando fallback para HTTP: {http_url}")
        try:
            response = session.get(http_url, timeout=timeout_seconds)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"Falha no fallback HTTP: {e}")
    return None

def get_all_csv_links_from_ckan():
    """Consulta a API CKAN e retorna todos os arquivos CSV da organização"""
    print("Consultando a API CKAN da CAPES...")
    params = {
        "fq": f"organization:{organization}",
        "rows": 1000
    }
    try:
        response = session.get(api_url, params=params, timeout=timeout_seconds)
        response.raise_for_status()
        results = response.json()["result"]["results"]
        csv_links = []
        for dataset in results:
            for resource in dataset.get("resources", []):
                if resource.get("format", "").lower() == "csv":
                    url = resource.get("url")
                    if url:
                        csv_links.append(url)
        return sorted(set(csv_links))
    except Exception as e:
        print(f"Erro ao consultar CKAN: {e}")
        return []

def detect_subfolder(filename):
    """Detecta subpasta com base nas substrings do nome do arquivo"""
    name = filename.lower()
    for prefix, substrings in prefix_substring_dirs.items():
        for substr, folder in substrings.items():
            target = prefix + substr
            if target in name:
                return folder
    return "outros"


def download_csv_files(links):
    """Baixa todos os arquivos CSV e organiza em subpastas por tipo"""
    total = len(links)
    print(f"\nIniciando download de {total} arquivos...")

    for i, url in enumerate(links, 1):
        try:
            raw_filename = url.split('/')[-1].split('?')[0]
            filename = sanitize_filename(raw_filename)
            subfolder = detect_subfolder(filename)

            subdir_path = os.path.join(output_dir, subfolder)
            os.makedirs(subdir_path, exist_ok=True)

            filepath = os.path.join(subdir_path, filename)

            if os.path.exists(filepath):
                print(f"[{i}/{total}] Já existe: {subfolder}/{filename}")
                continue

            print(f"[{i}/{total}] Baixando: {filename} para {subfolder}/")

            start_time = time.time()
            response = get_with_retry(url)
            if not response:
                print(f"Falha ao baixar {url}")
                continue

            with open(filepath, 'wb') as f:
                f.write(response.content)

            size_mb = os.path.getsize(filepath) / (1024 * 1024)
            print(f"Salvo como {subfolder}/{filename} ({size_mb:.2f} MB) em {time.time() - start_time:.2f}s")

        except Exception as e:
            print(f"Erro ao baixar {url}: {e}")

In [ ]:
#Código para download dos dados abertos - Processo potencialmente demorado
print("Iniciando processo para baixar todos os CSVs da Diretoria de Avaliação...")
start_time = time.time()
try:
    csv_links = get_all_csv_links_from_ckan()
    print(f"\n{len(csv_links)} arquivos CSV encontrados:")
    for i, link in enumerate(csv_links[:10], 1):
        print(f"{i}. {link.split('/')[-1].split('?')[0]}")
    if len(csv_links) > 10:
        print(f"... mais {len(csv_links) - 10} arquivos")
    if csv_links:
        download_csv_files(csv_links)
    print(f"\nConcluído em {time.time() - start_time:.2f}s")
except KeyboardInterrupt:
    print("\nProcesso interrompido pelo usuário.")
except Exception as e:
    print(f"Erro inesperado: {e}")

## Filtrando apenas registros da UFRJ dos dados abertos CAPES - 2013 em diante

In [3]:
#Função para selecionar documentos csv via expressões regulares e após um dado ano (e.g. 2013)
#A identificação do ano através do nome de arquivo desta função está adaptada à padronização de nomes dos arquivos da CAPES
#Valores de anos que iniciam quadriênios (2013, 2017, 2021) funcionarão normalmente. Outros anos poderão apresentar problemas
def filtrar_csvs_por_diretorio_regex_e_ano(
    caminho_do_diretorio,
    padroes_regex=None,
    ano_minimo=None, 
    busca_recursiva=False
):
    """
    Lista arquivos CSV em um diretório e os filtra com base em:
    1. Padrões de expressão regular (regex)
    2. Um ano mínimo encontrado no nome do arquivo.

    Args:
        caminho_do_diretorio (str): O caminho para o diretório base onde os CSVs estão.
        padroes_regex (list, optional): Uma lista de strings de expressões regulares.
                                        Arquivos devem corresponder a *qualquer um* desses padrões.
                                        Default é None (não aplica filtro regex).
        ano_minimo (int, optional): O ano mínimo (4 dígitos) para filtrar.
                                    Captura o primeiro grupo de 4 dígitos encontrado no nome do arquivo.
                                    Default é None (não aplica filtro de ano).
        busca_recursiva (bool, optional): Se True, a função buscará CSVs em subpastas também.
                                          Default é False.

    Returns:
        list: Uma lista de caminhos completos para os arquivos CSV que atendem a todos os critérios.

    Raises:
        ValueError: Se 'caminho_do_diretorio' não for encontrado ou se nenhum filtro for especificado.
    """
    if not os.path.exists(caminho_do_diretorio):
        print(f"Erro: O diretório '{caminho_do_diretorio}' não foi encontrado.")
        return []

    if not padroes_regex and ano_minimo is None:
        print("Aviso: Nenhum filtro (regex ou ano mínimo) foi especificado. Retornando todos os CSVs.")

    todos_csvs_encontrados = []

    # 1. Obter todos os arquivos CSV do diretório (e subpastas, se recursivo)
    if busca_recursiva:
        for root, _, files in os.walk(caminho_do_diretorio):
            for nome_arquivo in files:
                if nome_arquivo.lower().endswith(".csv"):
                    todos_csvs_encontrados.append(os.path.join(root, nome_arquivo))
    else:
        for nome_arquivo in os.listdir(caminho_do_diretorio):
            caminho_completo = os.path.join(caminho_do_diretorio, nome_arquivo)
            if os.path.isfile(caminho_completo) and nome_arquivo.lower().endswith(".csv"):
                todos_csvs_encontrados.append(caminho_completo)

    arquivos_filtrados_parcial = todos_csvs_encontrados

    # 2. Aplicar filtro por padrões Regex (se fornecidos)
    if padroes_regex:
        final_regex_filter = []
        regexes_compilados = [re.compile(p, re.IGNORECASE) for p in padroes_regex]

        for caminho_completo in arquivos_filtrados_parcial:
            nome_base = os.path.basename(caminho_completo)
            for regex in regexes_compilados:
                if regex.search(nome_base):
                    final_regex_filter.append(caminho_completo)
                    break
        arquivos_filtrados_parcial = final_regex_filter

    # 3. Aplicar filtro por ano mínimo (se fornecido)
    if ano_minimo is not None:
        final_year_filter = []
        # Regex para qualquer sequência de 4 dígitos - adaptado à padronização CAPES
        # O padrão (\d{4}) captura o ano. Valores de anos que iniciam quadriênios (2013, 2017, 2021)
        # funcionarão normalmente. Outros anos poderão apresentar problemas se o formato 'discentes-AAAA'
        # não for o primeiro e único lugar onde o ano pode estar.
        padrao_ano_capes = re.compile(r'(\d{4})') 

        for caminho_completo in arquivos_filtrados_parcial:
            nome_base = os.path.basename(caminho_completo)
            match = padrao_ano_capes.search(nome_base)
            
            if match:
                ano_str = match.group(1)
                try:
                    ano_int = int(ano_str)
                    if ano_int >= ano_minimo:
                        final_year_filter.append(caminho_completo)
                except ValueError:
                    print(f"Aviso: Não foi possível converter '{ano_str}' em ano inteiro para '{nome_base}'. Ignorando.")
        arquivos_filtrados_parcial = final_year_filter

    return arquivos_filtrados_parcial

In [4]:
#Função para obter caminhos utilizando o dicionário de diretórios definido no começo deste documento
def obter_caminho_completo(basedir_key, subdir_key, dirs=dirs):
    """
    Concatena os caminhos correspondentes a 'basedir_key' e 'subdir_key'
    do dicionário 'dirs' para formar um caminho completo.

    Args:
        basedir_key (str): A chave do diretório base em 'dirs'.
        subdir_key (str): A chave do subdiretório em 'dirs'.
        dirs (dict): Dicionário com o nome dos diretórios. Default: dirs.

    Returns:
        str: O caminho completo concatenado.

    Raises:
        ValueError: Se 'basedir_key' ou 'subdir_key' não forem chaves válidas em 'dirs'.
    """
    chaves_disponiveis = list(dirs.keys())

    if basedir_key not in dirs:
        raise ValueError(
            f"Erro: Chave '{basedir_key}' não encontrada em 'dirs' para basedir. "
            f"Chaves disponíveis: {chaves_disponiveis}"
        )

    if subdir_key not in dirs:
        raise ValueError(
            f"Erro: Chave '{subdir_key}' não encontrada em 'dirs' para subdir. "
            f"Chaves disponíveis: {chaves_disponiveis}"
        )
    
    # Obtém os valores de diretório do dicionário
    base_path = dirs[basedir_key]
    sub_path = dirs[subdir_key]

    # Concatena os caminhos usando os.path.join para compatibilidade entre sistemas
    caminho_final = os.path.join(base_path, sub_path)
    
    return caminho_final

In [5]:
#Junção das funções 'filtrar_csvs_por_diretorio_regex_e_ano()' e 'obter_caminho_completo()'
#Usada para obter uma lista com os arquivos csv de interesse a serem unificados em uma tabela única
def selecionar_csvs_capes(
    basedir_key,
    subdir_key,
    dirs=dirs,
    padroes_regex=None,
    ano_minimo=2013,
    busca_recursiva=False
):
    """
    Integra as funções para obter o caminho completo do diretório e filtrar arquivos CSV.

    Args:
        dirs (dict): Dicionário com o nome dos diretórios. Default: dirs.
        basedir_key (str): A chave do diretório base em 'dirs' (e.g., 'download_dir', 'ufrj_dir').
        subdir_key (str): A chave do subdiretório em 'dirs' (e.g., 'discentes', 'producao').
        padroes_regex (list, optional): Uma lista de strings de expressões regulares para filtrar nomes de arquivo.
                                        Arquivos devem corresponder a *qualquer um* desses padrões.
                                        Default é None (não aplica filtro regex).
        ano_minimo (int, optional): O ano mínimo (4 dígitos) para filtrar.
                                    Adapta-se ao padrão "discentes-AAAA" dos arquivos da CAPES.
                                    Valores de anos que iniciam quadriênios (2013, 2017, 2021) funcionarão
                                    normalmente. Outros anos poderão apresentar problemas se o formato não se encaixar.
                                    Default é None (não aplica filtro de ano).
        busca_recursiva (bool, optional): Se True, a função buscará CSVs em subpastas do diretório gerado.
                                          Default é False.

    Returns:
        list: Uma lista de caminhos completos para os arquivos CSV que atendem a todos os critérios.

    Raises:
        ValueError: Se 'basedir_key' ou 'subdir_key' forem inválidas, ou se o diretório final não existir.
    """
    try:
        # 1. Obter o caminho completo do diretório usando as chaves
        caminho_do_diretorio_completo = obter_caminho_completo(basedir_key, subdir_key)
        print(f"Buscando arquivos no diretório: {caminho_do_diretorio_completo}")

    except ValueError as e:
        print(f"Erro ao obter caminho do diretório: {e}")
        return []

    # 2. Filtrar os arquivos CSV dentro do diretório gerado
    arquivos_selecionados = filtrar_csvs_por_diretorio_regex_e_ano(
        caminho_do_diretorio=caminho_do_diretorio_completo,
        padroes_regex=padroes_regex,
        ano_minimo=ano_minimo,
        busca_recursiva=busca_recursiva
    )

    return arquivos_selecionados

In [6]:
#Função para processar e salvar csvs relacionados em um único arquivo
def fundir_lista_csvs(
        lista_caminhos_csv,
        colunas_desejadas,
        condicao_filtro_funcao,
        diretorio_saida,
        nome_arquivo_saida="saida_otimizada_lista.csv",
        chunk_size=10000,
        colunas_para_int64=None
        ):
    """
    Processa uma lista de arquivos CSV, extraindo colunas e linhas específicas de forma otimizada para RAM
    usando leitura em chunks. Inclui opção para converter colunas para tipo Int64 (inteiro com nulos)
    e lida com colunas ausentes em arquivos CSV.

    Args:
        lista_caminhos_csv (list): Uma lista de strings, onde cada string é o caminho completo para um arquivo CSV.
        colunas_desejadas (list): Uma lista de nomes de colunas a serem extraídas.
                                   A coluna usada para o filtro (se houver) deve ser incluída explicitamente aqui
                                   se você quiser que ela apareça no resultado final.
        condicao_filtro_funcao (function): Uma função que recebe uma linha (como Series do pandas)
                                           e retorna True se a linha deve ser incluída, False caso contrário.
        diretorio_saida (str): O caminho para o diretório onde o arquivo de saída será salvo.
        nome_arquivo_saida (str): O nome do arquivo CSV de saída (ex: "meu_arquivo.csv").
        chunk_size (int): O número de linhas a serem lidas por vez de cada arquivo CSV.
        colunas_para_int64 (list, optional): Uma lista de nomes de colunas que devem ser convertidas
                                              para o tipo 'Int64' (inteiro com suporte a nulos) antes de salvar.
                                              Isso evita a adição de '.0' em IDs. Default é None.
    """
    primeiro_arquivo = True
    coluna_filtro = None # Manter para compatibilidade, mas não será mais usado para adicionar/remover colunas

    caminho_completo_saida = os.path.join(diretorio_saida, nome_arquivo_saida)

    if diretorio_saida and not os.path.exists(diretorio_saida):
        os.makedirs(diretorio_saida, exist_ok=True)
        print(f"Diretório de saída criado: {diretorio_saida}")

    # A lógica de modificação de colunas_desejadas e colunas_para_salvar foi removida
    colunas_para_ler = list(colunas_desejadas) # Agora 'colunas_para_ler' é simplesmente 'colunas_desejadas'
    colunas_para_salvar = list(colunas_desejadas) # E 'colunas_para_salvar' também

    # A lógica de remoção da coluna de filtro também foi removida
    # if remove_filter_col e a lógica associada não estão mais presentes

    for caminho_completo_arquivo_entrada in lista_caminhos_csv:
        if not os.path.exists(caminho_completo_arquivo_entrada):
            print(f"Aviso: Arquivo não encontrado - {caminho_completo_arquivo_entrada}. Pulando...")
            continue
        if not caminho_completo_arquivo_entrada.lower().endswith(".csv"):
            print(f"Aviso: Ignorando arquivo não CSV - {caminho_completo_arquivo_entrada}.")
            continue

        print(f"Processando {caminho_completo_arquivo_entrada}...")

        # Ler o cabeçalho para verificar as colunas presentes
        try:
            df_header = pd.read_csv(caminho_completo_arquivo_entrada, sep=';', encoding='latin1', nrows=0)
            colunas_presentes_no_arquivo = df_header.columns.tolist()
        except Exception as e:
            print(f"Erro ao ler o cabeçalho do arquivo {caminho_completo_arquivo_entrada}: {e}. Pulando...")
            continue

        # Identificar colunas a serem lidas que realmente existem no arquivo
        colunas_a_realmente_ler = [col for col in colunas_para_ler if col in colunas_presentes_no_arquivo]
        colunas_ausentes_neste_arquivo = [col for col in colunas_para_ler if col not in colunas_presentes_no_arquivo]

        if colunas_ausentes_neste_arquivo:
            print(f"Aviso: As seguintes colunas desejadas não foram encontradas em '{caminho_completo_arquivo_entrada}': {', '.join(colunas_ausentes_neste_arquivo)}. Elas serão adicionadas como vazias.")

        read_csv_args = {
            'sep': ';',
            'encoding': 'latin1',
            'usecols': colunas_a_realmente_ler,
            'chunksize': chunk_size
        }

        try:
            for chunk in pd.read_csv(caminho_completo_arquivo_entrada, **read_csv_args):
                # Adicionar colunas ausentes no chunk com valores NaN (vazios)
                for col in colunas_ausentes_neste_arquivo:
                    chunk[col] = pd.NA

                df_filtrado = chunk[chunk.apply(condicao_filtro_funcao, axis=1)]

                # Certificar-se de que todas as colunas desejadas estão presentes antes de selecionar
                # e manter a ordem das colunas definidas em colunas_para_salvar
                df_final = pd.DataFrame(columns=colunas_para_salvar)
                for col in colunas_para_salvar:
                    if col in df_filtrado.columns:
                        df_final[col] = df_filtrado[col]
                    else:
                        df_final[col] = pd.NA

                # NOVO PASSO: Converter as colunas especificadas para Int64Dtype
                if colunas_para_int64:
                    for col in colunas_para_int64:
                        if col in df_final.columns:
                            try:
                                df_final[col] = pd.to_numeric(df_final[col], errors='coerce')
                                df_final[col] = df_final[col].astype('Int64')
                            except Exception as e:
                                print(f"Aviso: Não foi possível converter a coluna '{col}' para Int64 no chunk. Erro: {e}")
                        else:
                            print(f"Aviso: Coluna '{col}' não encontrada no DataFrame para conversão para Int64 neste chunk.")

                if primeiro_arquivo:
                    df_final.to_csv(caminho_completo_saida, mode='w', index=False)
                    primeiro_arquivo = False
                else:
                    df_final.to_csv(caminho_completo_saida, mode='a', header=False, index=False)
        except Exception as e:
            print(f"Erro ao ler ou processar o arquivo {caminho_completo_arquivo_entrada} em chunks: {e}")
            continue

    print(f"Processamento concluído. Saída salva em {caminho_completo_saida}")

### Discentes 

In [9]:
#Definição de filtro 
def filtro_ufrj_sigla(linha):
    """
    Verifica se a linha atende aos critérios de filtro:
    - 'SG_ENTIDADE_ENSINO' é 'UFRJ'.

    Args:
        linha (pd.Series): Uma linha do DataFrame.

    Returns:
        bool: True se a linha atende aos critérios, False caso contrário.
    """
    condicao_entidade = linha['SG_ENTIDADE_ENSINO'].strip().upper() == 'UFRJ'

    return condicao_entidade 

In [10]:
#Obtendo lista de arquivos csv com ano de referencia = 2013 ou maior (sem filtragem por regex)
discentes_csvs = selecionar_csvs_capes('download_dir', 'discentes')

Buscando arquivos no diretório: capes_csv_files/discentes


In [11]:
discentes_csvs

['capes_csv_files/discentes/br-capes-colsucup-discentes-2021-2025-03-31.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2013-2021-03-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2022-2025-03-31.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2014-2021-03-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2020-2023-12-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2016-2021-03-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2023-2025-03-31.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2018-2023-12-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2015-2021-03-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2019-2023-12-01.csv',
 'capes_csv_files/discentes/br-capes-colsucup-discentes-2017-2023-12-01.csv']

In [12]:
fundir_lista_csvs(discentes_csvs, 
                colunas_desejadas=['AN_BASE', 'ID_PESSOA', 'CD_PROGRAMA_IES', 'NM_DISCENTE', 'DS_TIPO_NACIONALIDADE_DISCENTE', 
                                    'NM_PAIS_NACIONALIDADE_DISCENTE', 'AN_NASCIMENTO_DISCENTE',
                                    'DS_GRAU_ACADEMICO_DISCENTE', 'ST_INGRESSANTE', 'NM_SITUACAO_DISCENTE',
                                    'QT_MES_TITULACAO', 'SG_ENTIDADE_ENSINO'], 
                                    condicao_filtro_funcao=filtro_ufrj_sigla,
                                    diretorio_saida=filtered_dir,
                                    nome_arquivo_saida='discentes.csv'
                )

Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2021-2025-03-31.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2013-2021-03-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2022-2025-03-31.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2014-2021-03-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2020-2023-12-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2016-2021-03-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2023-2025-03-31.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2018-2023-12-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2015-2021-03-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2019-2023-12-01.csv...
Processando capes_csv_files/discentes/br-capes-colsucup-discentes-2017-2023-12-01.csv...
Processamento concluí

### Docentes

In [13]:
docentes_csvs = selecionar_csvs_capes('download_dir', 'docentes')

Buscando arquivos no diretório: capes_csv_files/docentes


In [14]:
docentes_csvs

['capes_csv_files/docentes/br-capes-colsucup-docente-2023-2025-03-31.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2013-2023-08-01.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2019-2021-11-10.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2022-2025-03-31.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2015-2023-08-01.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2020-2021-11-10.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2017-2021-11-10.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2021-2025-03-31.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2016-2023-08-01.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2014-2023-08-01.csv',
 'capes_csv_files/docentes/br-capes-colsucup-docente-2018-2021-11-10.csv']

In [15]:
def filtro_docente(linha):
    """
    Verifica se a linha atende aos critérios de filtro:
    - 'SG_ENTIDADE_ENSINO' é 'UFRJ'.
    - 'DS_CATEGORIA_DOCENTE' é 'PERMANENTE'

    Args:
        linha (pd.Series): Uma linha do DataFrame.

    Returns:
        bool: True se a linha atende aos critérios, False caso contrário.
    """
    condicao_entidade = linha['SG_ENTIDADE_ENSINO'].strip().upper() == 'UFRJ'
    condicao_categoria_docente = linha['DS_CATEGORIA_DOCENTE'].strip().upper() == 'PERMANENTE'

    return condicao_entidade and condicao_categoria_docente

In [16]:
fundir_lista_csvs(docentes_csvs, 
                  colunas_desejadas=['AN_BASE', 'ID_PESSOA', 'CD_PROGRAMA_IES',
                  'NM_DOCENTE', 'AN_NASCIMENTO_DOCENTE', 'DS_TIPO_NACIONALIDADE_DOCENTE',
                  'NM_PAIS_NACIONALIDADE_DOCENTE', 'DS_CATEGORIA_DOCENTE', 
                  'DS_TIPO_VINCULO_DOCENTE_IES', 'DS_REGIME_TRABALHO',
                  'CD_CAT_BOLSA_PRODUTIVIDADE', 'NM_GRAU_TITULACAO', 'SG_ENTIDADE_ENSINO'], 
                  condicao_filtro_funcao=filtro_docente,
                  diretorio_saida=filtered_dir,
                  nome_arquivo_saida='docentes.csv')

Processando capes_csv_files/docentes/br-capes-colsucup-docente-2023-2025-03-31.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2013-2023-08-01.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2019-2021-11-10.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2022-2025-03-31.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2015-2023-08-01.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2020-2021-11-10.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2017-2021-11-10.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2021-2025-03-31.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2016-2023-08-01.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2014-2023-08-01.csv...
Processando capes_csv_files/docentes/br-capes-colsucup-docente-2018-2021-11-10.csv...
Processamento concluído. Saída salva em ufrj_data/doce

### Programas

In [ ]:
programas_csvs = selecionar_csvs_capes('download_dir', 'programas', padroes_regex=[r"br-capes-colsucup-prog"])

In [ ]:
programas_csvs

In [ ]:
fundir_lista_csvs(programas_csvs, ['AN_BASE', 'CD_PROGRAMA_IES', 'NM_PROGRAMA_IES', 'NM_GRANDE_AREA_CONHECIMENTO', 
                                    'NM_GRAU_PROGRAMA', 'CD_CONCEITO_PROGRAMA', 'ANO_INICIO_PROGRAMA', 'AN_INICIO_PROGRAMA',
                                    'AN_INICIO_CURSO', 'IN_REDE', 'DS_SITUACAO_PROGRAMA',
                                    'CD_AREA_AVALIACAO', 'NM_AREA_AVALIACAO', 'NM_MODALIDADE_PROGRAMA', 'SG_ENTIDADE_ENSINO',
                                   ], 
                                   condicao_filtro_funcao=filtro_ufrj_sigla,
                                   diretorio_saida=filtered_dir,
                                   nome_arquivo_saida='programas.csv'
                                   )

### Produção (artigos de periódicos por autor)

In [ ]:
producao_csvs = selecionar_csvs_capes('download_dir', 'producao', padroes_regex=[r"br-capes-colsucup-prod-autor-.*bibliografica-artpe"])

In [ ]:
producao_csvs

In [ ]:
def filtro_producao(linha):
    """
    Verifica se a linha atende aos critérios de filtro:
    - 'SG_ENTIDADE_ENSINO' é 'UFRJ'.
    - 'ID_PESSOA_DOCENTE' não é nulo.
    - 'ID_PESSOA_DISCENTE' não é nulo.
    - 'NM_TP_CATEGORIA_DOCENTE' é 'PERMANENTE'.
    - 'NM_NIVEL_DISCENTE' está em algum nível que corresponda a alunos de pós.

    Args:
        linha (pd.Series): Uma linha do DataFrame.

    Returns:
        bool: True se a linha atende aos critérios, False caso contrário.
    """
    condicao_entidade = linha['SG_ENTIDADE_ENSINO'].strip().upper() == 'UFRJ'
    condicao_discente_nao_nulo = pd.notna(linha['ID_PESSOA_DISCENTE'])
    condicao_docente_nao_nulo = pd.notna(linha['ID_PESSOA_DOCENTE'])

    condicao_categoria_docente = True #Se a linha não estiver se referindo a docente, isso deve ser verdadeiro para não excluir o registro
    condicao_nivel_discente = True #Se a linha não estiver se referindo a discente, isso deve ser verdadeiro para não excluir o registro

    if condicao_docente_nao_nulo:
        nm_categoria_docente = str(linha.get('NM_TP_CATEGORIA_DOCENTE', '')).strip().upper()
        condicao_categoria_docente = (nm_categoria_docente == 'PERMANENTE') #Só inclui docentes permanentes
    elif condicao_discente_nao_nulo:
        nm_nivel_discente = str(linha.get('NM_NIVEL_DISCENTE', '')).strip().upper()
        condicao_nivel_discente = (nm_nivel_discente in ['MESTRADO', 'DOUTORADO', 'MESTRADO PROFISSIONAL', 'DOUTORADO PROFISSIONAL']) #Só inclui alunos de pós 
    
    return condicao_entidade and (condicao_discente_nao_nulo or condicao_docente_nao_nulo) and condicao_categoria_docente and condicao_nivel_discente

In [ ]:
fundir_lista_csvs(producao_csvs, 
                colunas_desejadas=['AN_BASE', 'CD_PROGRAMA_IES', 'ID_ADD_PRODUCAO_INTELECTUAL', 
                                   'ID_PESSOA_DOCENTE', 'ID_PESSOA_DISCENTE', 'TP_AUTOR', 
                                   'NM_TP_CATEGORIA_DOCENTE', 'NM_NIVEL_DISCENTE', 'SG_ENTIDADE_ENSINO'], 
                condicao_filtro_funcao=filtro_producao, 
                diretorio_saida=filtered_dir,
                nome_arquivo_saida='producao.csv',
                colunas_para_int64= ['ID_PESSOA_DOCENTE', 'ID_PESSOA_DISCENTE'] ,
               )

## Processamento/limpeza dos dados da ufrj

- Discentes:
    - Geração da coluna de gênero/remoção da coluna de nome
    - Converter ST_INGRESSANTE para booleano

- Docentes: 
    - Geração da coluna de gênero/remoção da coluna de nome
    - Limpeza campo CD_CAT_BOLSA_PRODUTIVIDADE (remover os NA e SR)

- Pessoas:
    - Pegar os campos de docentes e discentes que não mudam ao longo dos anos

- Programas:
    - Campo IN_REDE convertido para booleano
    - Substituir conceito 'A' ('Ausente') por 0
    - Fundir colunas ANO_INICIO_PROGRAMA (2013-2016) e AN_INICIO_PROGRAMA (2017 em diante)
    - Gerar a tabela 'programa' (informações que não mudam ao longo dos anos)
    - Gerar a tabela 'ano_programa', contendo apenas as informações sobre os programas que podem mudar ao longo dos anos
    - Gerar a tabela 'cursos', com a relação entre CD_PROGRAMA_IES, NM_GRAU_PROGRAMA e ANO_INICIO_CURSO
        - Separar os valores separados por barra em NM_GRAU_PROGRAMA (e.g. MESTRADO/DOUTORADO) e ANO_INICIO_CURSO (e.g. 1981/2001)
        - Adicionar outras linhas normalmente

- Produção:
    - Unir as colunas ID_PESSOA_DOCENTE e ID_PESSOA_DISCENTE em uma só

- Todas:
    - Remover as colunas usadas para a filtragem dos dados

In [17]:
#Importando dicionário de gêneros
with open('aux/dicionario_generos.json', 'r') as f: 
    dicionario_generos = json.load(f)

dicionario_generos

{'AALINE': 'F',
 'AILINE': 'F',
 'ALEINE': 'F',
 'ALIINE': 'F',
 'ALINE': 'F',
 'ALINER': 'F',
 'ALINHE': 'F',
 'ALINNE': 'F',
 'ALYNE': 'F',
 'ALYNNE': 'F',
 'AYLINE': 'F',
 'EALINE': 'F',
 'ELEINE': 'F',
 'ELINE': 'F',
 'ELINER': 'F',
 'ELINNE': 'F',
 'ELYNE': 'F',
 'EULINE': 'F',
 'HALINE': 'F',
 'HALYNE': 'F',
 'HELEINE': 'F',
 'HELINE': 'F',
 'HELYNE': 'F',
 'IALINE': 'F',
 'ILEINE': 'F',
 'ILINE': 'F',
 'LEINE': 'F',
 'LEINER': 'F',
 'LEYNE': 'F',
 'LINE': 'F',
 'LINER': 'F',
 'LUEINE': 'F',
 'LUINE': 'F',
 'LUYNE': 'F',
 'LYNE': 'F',
 'LYNNE': 'F',
 'OLINE': 'F',
 'UELINE': 'F',
 'AARAO': 'M',
 'ARAAO': 'M',
 'ARAO': 'M',
 'AARON': 'M',
 'AHARON': 'M',
 'AROM': 'M',
 'ARON': 'M',
 'ARYON': 'M',
 'HARON': 'M',
 'ABA': 'F',
 'ADA': 'F',
 'ADAH': 'F',
 'ADAR': 'F',
 'ADHA': 'F',
 'HADA': 'F',
 'ABADE': 'M',
 'ABADI': 'M',
 'ABADIR': 'M',
 'ABADIA': 'F',
 'ABADIAS': 'M',
 'ABADIO': 'M',
 'ABAETE': 'F',
 'ABETE': 'F',
 'ADETE': 'F',
 'ABD': 'M',
 'ABDA': 'F',
 'ADDA': 'F',
 'ABDAEL':

In [18]:
# Define a função auxiliar que será aplicada a cada nome completo
def genero_baseado_no_primeiro_nome(nome_completo: str,
                    dicionario_generos: dict) -> str:
    
    # Garante que é uma string, útil para lidar com NaNs ou outros tipos
    nome_completo_str = str(nome_completo) 
    
    primeiro_nome = nome_completo_str.split(' ')[0].strip().upper()
    
    # Usa .get() para retornar 'D' (Desconhecido) se o nome não for encontrado
    return dicionario_generos.get(primeiro_nome, 'D')


def adicionar_coluna_genero(df,
                                 coluna_nome_completo: str, 
                                 dicionario_generos: dict,
                                 coluna_genero: str = 'GN_PESSOA',
                       ):
    """
    Extrai o primeiro nome de uma coluna, consulta um dicionário de gêneros baseados em primeiros nomes
    e retorna os gêneros correspondentes em uma nova coluna, usando df.apply().

    Args:
        df (pd.DataFrame): O DataFrame de entrada.
        coluna_nome_completo (str): O nome da coluna no DataFrame que contém os nomes completos.
        dicionario_generos (dict): Um dicionário onde as chaves são os primeiros nomes (em maiúsculas)
                                   e os valores são os gêneros ('F', 'M', 'Desconhecido', etc.).
        coluna_genero (str): Nome da nova coluna com o genero inferido com base no primeiro nome. O padrão é 'GN_PESSOA'.

    Returns:
        pd.DataFrame: O DataFrame original com uma nova coluna .
    """
    df[coluna_genero] = df[coluna_nome_completo].apply(genero_baseado_no_primeiro_nome, 
                                                       dicionario_generos=dicionario_generos)
    
    return df

In [19]:
#Criando diretório de saída dos arquivos processados
try:
    # Cria o diretório
    # 'exist_ok=True' é crucial: se o diretório já existir, ele não levantará um erro (FileExistsError)
    os.makedirs(processed_dir, exist_ok=True)
    print(f"Diretório '{processed_dir}' criado com sucesso ou já existente.")
except OSError as e:
    # Trata outros possíveis erros do sistema operacional (permissões, nomes inválidos, etc.)
    print(f"Erro ao criar o diretório '{processed_dir}': {e}")

Diretório 'sucupira_painel' criado com sucesso ou já existente.


### Discentes

In [20]:
#Importando dados dos discentes
discentes = pd.read_csv(f'{filtered_dir}/discentes.csv')
discentes.head()

,AN_BASE,ID_PESSOA,CD_PROGRAMA_IES,NM_DISCENTE,DS_TIPO_NACIONALIDADE_DISCENTE,NM_PAIS_NACIONALIDADE_DISCENTE,AN_NASCIMENTO_DISCENTE,DS_GRAU_ACADEMICO_DISCENTE,ST_INGRESSANTE,NM_SITUACAO_DISCENTE,QT_MES_TITULACAO,SG_ENTIDADE_ENSINO
0,2021,3353229,31001017100P2,BRENDO ARAUJO GOMES,BRASILEIRO,BRASIL,1994,DOUTORADO,SIM,MATRICULADO,0,UFRJ
1,2021,26505,31001017033P3,CLAUDIA BENITEZ LOGELO,BRASILEIRO,BRASIL,1973,DOUTORADO,NÃO,MATRICULADO,0,UFRJ
2,2021,1216819,31001017172P3,FRANCIANE PIMENTEL MELO,BRASILEIRO,BRASIL,1974,MESTRADO,NÃO,MATRICULADO,0,UFRJ
3,2021,794525,31001017020P9,GABRIELA MONTEZ HOLANDA DA SILVA,BRASILEIRO,BRASIL,1990,DOUTORADO,NÃO,MATRICULADO,0,UFRJ
4,2021,4485640,31001017134P4,DANIELLE BRODA DE VASCONCELLOS,BRASILEIRO,BRASIL,1991,MESTRADO PROFISSIONAL,SIM,MATRICULADO,0,UFRJ


In [21]:
#Gerando a coluna de gênero com base no primeiro nome
discentes = adicionar_coluna_genero(discentes,
                        coluna_nome_completo='NM_DISCENTE',
                        dicionario_generos=dicionario_generos)
discentes[['NM_DISCENTE', 'GN_PESSOA']].head()

,NM_DISCENTE,GN_PESSOA
0,BRENDO ARAUJO GOMES,M
1,CLAUDIA BENITEZ LOGELO,F
2,FRANCIANE PIMENTEL MELO,F
3,GABRIELA MONTEZ HOLANDA DA SILVA,F
4,DANIELLE BRODA DE VASCONCELLOS,F


In [22]:
#Convertendo coluna ST_INGRESSANTE para booleano
mapeamento_booleano = {'SIM': True, 'NÃO': False}
discentes['ST_INGRESSANTE'] = discentes['ST_INGRESSANTE'].map(mapeamento_booleano)

In [23]:
#Mantendo apenas as colunas necessárias (que mudam ao longo do tempo)
discentes_final_cols = ['AN_BASE', 'ID_PESSOA', 'CD_PROGRAMA_IES', 'DS_GRAU_ACADEMICO_DISCENTE', 'ST_INGRESSANTE', 'NM_SITUACAO_DISCENTE', 'QT_MES_TITULACAO']
discentes_final = discentes[discentes_final_cols]

In [24]:
#Visualizando dataframe final
discentes_final.head()

,AN_BASE,ID_PESSOA,CD_PROGRAMA_IES,DS_GRAU_ACADEMICO_DISCENTE,ST_INGRESSANTE,NM_SITUACAO_DISCENTE,QT_MES_TITULACAO
0,2021,3353229,31001017100P2,DOUTORADO,True,MATRICULADO,0
1,2021,26505,31001017033P3,DOUTORADO,False,MATRICULADO,0
2,2021,1216819,31001017172P3,MESTRADO,False,MATRICULADO,0
3,2021,794525,31001017020P9,DOUTORADO,False,MATRICULADO,0
4,2021,4485640,31001017134P4,MESTRADO PROFISSIONAL,True,MATRICULADO,0


In [25]:
#Salvando dataframe final
discentes_final.to_csv(f'{processed_dir}/discentes.csv', index=False)

### Docentes

In [26]:
#Importando df filtrada
docentes = pd.read_csv(f'{filtered_dir}/docentes.csv')
docentes.head()

,AN_BASE,ID_PESSOA,CD_PROGRAMA_IES,NM_DOCENTE,AN_NASCIMENTO_DOCENTE,DS_TIPO_NACIONALIDADE_DOCENTE,NM_PAIS_NACIONALIDADE_DOCENTE,DS_CATEGORIA_DOCENTE,DS_TIPO_VINCULO_DOCENTE_IES,DS_REGIME_TRABALHO,CD_CAT_BOLSA_PRODUTIVIDADE,NM_GRAU_TITULACAO,SG_ENTIDADE_ENSINO
0,2023,513065,31001017003P7,KATRIN GRIT GELFERT,1973,ESTRANGEIRO,ALEMANHA,PERMANENTE,SERVIDOR PÚBLICO,INTEGRAL,1C,DOUTORADO,UFRJ
1,2023,173737,31001017003P7,SERGIO AUGUSTO ROMANA IBARRA,1984,BRASILEIRO,BRASIL,PERMANENTE,SERVIDOR PÚBLICO,INTEGRAL,NaN,DOUTORADO,UFRJ
2,2023,1129065,31001017003P7,ISAIA NISOLI,1982,ESTRANGEIRO,ITÁLIA,PERMANENTE,SERVIDOR PÚBLICO,INTEGRAL,NaN,DOUTORADO,UFRJ
3,2023,939007,31001017003P7,SEYED HAMID HASSANZADEH HAFSHEJANI,1982,ESTRANGEIRO,IRÃ,PERMANENTE,SERVIDOR PÚBLICO,DEDICAÇÃO EXCLUSIVA,NaN,DOUTORADO,UFRJ
4,2023,535643,31001017003P7,LUCIANE QUOOS CONTE,1971,BRASILEIRO,BRASIL,PERMANENTE,SERVIDOR PÚBLICO,INTEGRAL,1D,DOUTORADO,UFRJ


In [27]:
#Gerando a coluna de gênero com base no primeiro nome
docentes = adicionar_coluna_genero(docentes,
                        coluna_nome_completo='NM_DOCENTE',
                        dicionario_generos=dicionario_generos)
docentes[['NM_DOCENTE', 'GN_PESSOA']].head()

,NM_DOCENTE,GN_PESSOA
0,KATRIN GRIT GELFERT,F
1,SERGIO AUGUSTO ROMANA IBARRA,M
2,ISAIA NISOLI,M
3,SEYED HAMID HASSANZADEH HAFSHEJANI,D
4,LUCIANE QUOOS CONTE,F


In [28]:
#limpeza do campo CD_CAT_PRODUTIVIDADE
docentes['CD_CAT_BOLSA_PRODUTIVIDADE'].unique()

array(['1C', nan, '1D', '1B', '1A', '2', 'SR'], dtype=object)

In [29]:
docentes['CD_CAT_BOLSA_PRODUTIVIDADE'] = docentes['CD_CAT_BOLSA_PRODUTIVIDADE'].replace([np.nan, 'SR'], pd.NA)
docentes['CD_CAT_BOLSA_PRODUTIVIDADE'].unique()

array(['1C', <NA>, '1D', '1B', '1A', '2'], dtype=object)

In [30]:
#Checando se só temos docentes permanentes
docentes['DS_CATEGORIA_DOCENTE'].unique()

array(['PERMANENTE'], dtype=object)

In [31]:
#Mantendo apenas colunas necessárias (que mudam ao longo do tempo)
docentes_final_cols = ['AN_BASE', 'ID_PESSOA', 'CD_PROGRAMA_IES', 'DS_TIPO_VINCULO_DOCENTE_IES', 'DS_REGIME_TRABALHO', 'CD_CAT_BOLSA_PRODUTIVIDADE', 'NM_GRAU_TITULACAO']
docentes_final = docentes[docentes_final_cols]

In [32]:
#Visualizando resultado final
docentes_final.head()

,AN_BASE,ID_PESSOA,CD_PROGRAMA_IES,DS_TIPO_VINCULO_DOCENTE_IES,DS_REGIME_TRABALHO,CD_CAT_BOLSA_PRODUTIVIDADE,NM_GRAU_TITULACAO
0,2023,513065,31001017003P7,SERVIDOR PÚBLICO,INTEGRAL,1C,DOUTORADO
1,2023,173737,31001017003P7,SERVIDOR PÚBLICO,INTEGRAL,<NA>,DOUTORADO
2,2023,1129065,31001017003P7,SERVIDOR PÚBLICO,INTEGRAL,<NA>,DOUTORADO
3,2023,939007,31001017003P7,SERVIDOR PÚBLICO,DEDICAÇÃO EXCLUSIVA,<NA>,DOUTORADO
4,2023,535643,31001017003P7,SERVIDOR PÚBLICO,INTEGRAL,1D,DOUTORADO


In [33]:
#Salvando dataframe final
docentes_final.to_csv(f'{processed_dir}/docentes.csv', index=False)

### Pessoas

In [34]:
def padronizar_nomes_colunas(df: pd.DataFrame) -> pd.DataFrame:
    """
    Renomeia colunas que terminam com '_DOCENTE' ou '_DISCENTE' para nomes padronizados.
    Ex: 'AN_NASCIMENTO_DOCENTE' -> 'AN_NASCIMENTO'
        'NM_PAIS_NACIONALIDADE_DISCENTE' -> 'NM_PAIS_NACIONALIDADE'

    Args:
        df (pd.DataFrame): O DataFrame a ser renomeado.

    Returns:
        pd.DataFrame: O DataFrame com as colunas renomeadas.
    """
    novo_mapeamento = {}
    for coluna in df.columns:
        if coluna.endswith('_DOCENTE'):
            novo_mapeamento[coluna] = coluna.replace('_DOCENTE', '')
        elif coluna.endswith('_DISCENTE'):
            novo_mapeamento[coluna] = coluna.replace('_DISCENTE', '')
        else:
            novo_mapeamento[coluna] = coluna # Mantém colunas sem sufixo inalteradas
    
    return df.rename(columns=novo_mapeamento)

In [ ]:
#Padronizando nomes de colunas entre docentes e discentes e extraindo colunas da tabela 'pessoas'
pessoas_cols = ['AN_BASE', 'ID_PESSOA', 'NM', 'AN_NASCIMENTO', 'DS_TIPO_NACIONALIDADE', 'NM_PAIS_NACIONALIDADE', 'GN_PESSOA']
pessoas_docentes = padronizar_nomes_colunas(docentes)[pessoas_cols]
pessoas_discentes = padronizar_nomes_colunas(discentes)[pessoas_cols]

In [38]:
pessoas = pd.concat([pessoas_discentes, pessoas_discentes])

In [39]:
pessoas

,AN_BASE,ID_PESSOA,NM,AN_NASCIMENTO,DS_TIPO_NACIONALIDADE,NM_PAIS_NACIONALIDADE,GN_PESSOA
0,2021,3353229,BRENDO ARAUJO GOMES,1994,BRASILEIRO,BRASIL,M
1,2021,26505,CLAUDIA BENITEZ LOGELO,1973,BRASILEIRO,BRASIL,F
2,2021,1216819,FRANCIANE PIMENTEL MELO,1974,BRASILEIRO,BRASIL,F
3,2021,794525,GABRIELA MONTEZ HOLANDA DA SILVA,1990,BRASILEIRO,BRASIL,F
4,2021,4485640,DANIELLE BRODA DE VASCONCELLOS,1991,BRASILEIRO,BRASIL,F
...,...,...,...,...,...,...,...
158386,2017,978622,RAIMUNDO GOMES BARBOSA,1956,BRASILEIRO,BRASIL,M
158387,2017,999691,OTAVIO CABRERA DE LEO,1975,BRASILEIRO,BRASIL,M
158388,2017,1001943,LETICIA VON KRUGER PIMENTEL,1973,BRASILEIRO,BRASIL,F
158389,2017,1030429,VERUSKA POBIKROWSKA TARDIVO,1977,BRASILEIRO,BRASIL,F


In [45]:
pessoas_sem_duplicatas = pessoas.drop_duplicates(['ID_PESSOA', 'NM', 'AN_NASCIMENTO', 'DS_TIPO_NACIONALIDADE', 'NM_PAIS_NACIONALIDADE', 'GN_PESSOA'])

In [46]:
pessoas_sem_duplicatas['ID_PESSOA'].duplicated().any()

np.True_

In [47]:
mascara_duplicatas = pessoas_sem_duplicatas['ID_PESSOA'].duplicated(keep=False)

In [48]:
pessoas_sem_duplicatas[mascara_duplicatas].sort_values(by='ID_PESSOA').to_csv(f'{processed_dir}/id_pessoa_dups.csv')

In [ ]:
print(docentes.columns.tolist())

In [ ]:
print(discentes.columns.tolist())

### Programas

In [ ]:
#Importando df filtrada
programas = pd.read_csv(f'{filtered_dir}/programas.csv')
programas

In [ ]:
#Checando valores do campo 'IN_REDE'
programas['IN_REDE'].unique()

In [ ]:
#Convertendo valores para booleans
mapeamento_booleano = {'SIM': True, 'NÃO': False}
programas['IN_REDE'] = programas['IN_REDE'].map(mapeamento_booleano)

In [ ]:
programas['IN_REDE'].unique()

In [ ]:
#Checando conceitos atribuídos às PPGs
programas['CD_CONCEITO_PROGRAMA'].unique()

In [ ]:
#Convertendo conceito 'A' (provavelmente 'Ausente') para 0
programas['CD_CONCEITO_PROGRAMA'] = programas['CD_CONCEITO_PROGRAMA'].replace('A', 0).astype('int') 

In [ ]:
programas['CD_CONCEITO_PROGRAMA'].unique()

In [ ]:
#Checando colunas ANO_INICIO_PROGRAMA e AN_INICIO_PROGRAMA 
print(programas[['ANO_INICIO_PROGRAMA', 'AN_INICIO_PROGRAMA']])

In [ ]:
#Fundindo colunas ANO_INICIO_PROGRAMA e AN_INICIO_PROGRAMA 
#'AN_INICIO_PROGRAMA' convertido para int para evitar a permanência de floats graças aos nan anteriores
programas['AN_INICIO_PROGRAMA'] = programas['AN_INICIO_PROGRAMA'].fillna(programas['ANO_INICIO_PROGRAMA']).astype('int') 

In [ ]:
print(programas[['ANO_INICIO_PROGRAMA', 'AN_INICIO_PROGRAMA']])

In [ ]:
#Removendo colunas desnecessárias
programas_final = programas.drop(columns = ['ANO_INICIO_PROGRAMA', 'SG_ENTIDADE_ENSINO', 'AN_INICIO_CURSO'])

In [ ]:
programas_final

In [ ]:
#Salvando dataframe programas_final
programas_final.to_csv(f'{processed_dir}/programas.csv', index=False)

In [ ]:
ano_programa = 

In [ ]:
#Agora, será necessário extrair a tabela 'cursos' de dentro da tabela 'programas'
#Lembrando que alguns programas oferecem mestrado e doutorado simultaneamente
# Logo, precisaremos de alguma lógica para lidar com isso
print(f'Valores únicos: {programas['NM_GRAU_PROGRAMA'].unique()}')
print()
print(programas[['NM_GRAU_PROGRAMA', 'AN_INICIO_CURSO']])

In [ ]:
def gerador_tabela_cursos(
    df_programas: pd.DataFrame,
    coluna_id_programa: str = 'CD_PROGRAMA_IES', 
    coluna_grau: str = 'NM_GRAU_PROGRAMA',
    coluna_ano: str = 'AN_INICIO_CURSO'
) -> pd.DataFrame:
    """
    Desagrega programas que possuem graus combinados (ex: 'MESTRADO/DOUTORADO')
    e anos de início correspondentes (ex: '1981/2001') em linhas separadas.

    Após a desagregação, a função retorna um novo DataFrame contendo apenas
    as colunas CD_PROGRAMA_IES (ou o nome fornecido), NM_GRAU_PROGRAMA (ou o nome fornecido),
    e AN_INICIO_CURSO (ou o nome fornecido), com entradas duplicadas removidas.

    Args:
        df_programas (pd.DataFrame): O DataFrame de entrada.
        coluna_id_programa (str): O nome da coluna que contém o código do programa (ex: 'CD_PROGRAMA_IES').
        coluna_grau (str): O nome da coluna que contém o grau do programa (padrão: 'NM_GRAU_PROGRAMA').
        coluna_ano (str): O nome da coluna que contém o ano de início do curso (padrão: 'AN_INICIO_CURSO').

    Returns:
        pd.DataFrame: Um novo DataFrame com as colunas especificadas, graus desagregados
                      e entradas duplicadas removidas.

    Raises:
        ValueError: Se as colunas especificadas não existirem no DataFrame.
    """
    # Garante que as colunas existem no DataFrame de entrada
    colunas_necessarias = [coluna_id_programa, coluna_grau, coluna_ano]
    for col in colunas_necessarias:
        if col not in df_programas.columns:
            raise ValueError(f"A coluna '{col}' não foi encontrada no DataFrame. Verifique o nome da coluna.")

    # Trabalha em uma cópia para não modificar o DataFrame original
    df_trabalho = df_programas.copy()

    # Identifica as linhas que contêm '/' na coluna de grau especificada
    mascara_combinados = df_trabalho[coluna_grau].str.contains('/', na=False)
    programas_combinados = df_trabalho[mascara_combinados].copy()
    programas_outros_graus = df_trabalho[~mascara_combinados].copy()

    df_novas_linhas = pd.DataFrame() # DataFrame vazio para acumular as novas linhas geradas

    if not programas_combinados.empty:
        # Divide tanto a coluna de graus quanto a de anos, usando os nomes passados
        graus_divididos = programas_combinados[coluna_grau].str.split('/', expand=True)
        anos_divididos = programas_combinados[coluna_ano].str.split('/', expand=True)

        # Itera sobre as possíveis "partes"
        for i in range(max(len(graus_divididos.columns), len(anos_divididos.columns))):
            nome_grau_parte = graus_divididos.get(i)
            ano_curso_parte = anos_divididos.get(i)

            if nome_grau_parte is not None and ano_curso_parte is not None:
                df_parte = programas_combinados.copy()

                df_parte[coluna_grau] = nome_grau_parte
                df_parte[coluna_ano] = ano_curso_parte

                df_novas_linhas = pd.concat([df_novas_linhas, df_parte], ignore_index=True)

        df_novas_linhas = df_novas_linhas.dropna(subset=[coluna_ano])

    # Concatena o DataFrame de programas sem combinação e as novas linhas geradas
    programas_final_completo = pd.concat([programas_outros_graus, df_novas_linhas], ignore_index=True)

    # --- Nova Lógica para selecionar colunas e remover duplicatas ---

    # Seleciona apenas as colunas desejadas
    df_resultado = programas_final_completo[[coluna_id_programa, coluna_grau, coluna_ano]].copy()

    # Opcional: Converte a coluna de ano de início para tipo numérico
    df_resultado[coluna_ano] = pd.to_numeric(
        df_resultado[coluna_ano], errors='coerce'
    ).astype('Int64') # Usando Int64 para aceitar nulos, como discutido anteriormente

    # Remove entradas duplicadas com base nas três colunas
    df_resultado.drop_duplicates(inplace=True)

    return df_resultado

In [ ]:
cursos = gerador_tabela_cursos(programas) #Pode usar a tabela inicial, já que ela vai ter todas as informações

In [ ]:
cursos

In [ ]:
programas_fulltable = desagregar_graus_programa(programas_clean)

In [ ]:
programas_fulltable

### Produção

In [ ]:
#Importando df filtrada
producao = pd.read_csv(f'{filtered_dir}/producao.csv')
producao

In [ ]:
producao['NM_NIVEL_DISCENTE'].unique()

In [ ]:
producao[producao['NM_NIVEL_DISCENTE'] == 'DOUTORADO'][['TP_AUTOR', 'NM_NIVEL_DISCENTE']]